## Check List 4 - PlanningPipeline
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4.1 PlanningScene**  
  - create_binder
    - actor: PlacePlane, Gripper2Tool, SweepTool
    - handle: Grasp2Point, PlacePoint, SweepPoint
  - create_subject
    - subject: CustomObject, SweepLineTask
  - initialize_state
* **4.2 MotionPlanner**  
  - MoveitPlanner
    - pick, place, sweep
    - Interpolated path generation by incremental IK
* **4.3 TaskPlanner**  
  - RRT
    - pick & place & sweep
      - single process
      - multi process
  - PDDLStream
    - pick & place

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=np.random.rand(3)*0, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
#                            rpy=np.random.rand(3)*np.pi/16, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

In [7]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.04,0.02,0.1), center=(0,0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.04,0.02,0.1), center=(0,-0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

In [8]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepFramer, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [12]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('floor', 0)


## 4.2 MotionPlanner

##### MoveitPlanner
* *get_available_binding_dict()* gets available binding states for each subject in a dictionary
* *sample_leaf_state samples()* target state with given available_binding_dict and target node
* *rebind_all()* updates binding state and returns the resultant state
* The motions tested in this section are:
  - pick: move the object to "gripper"
  - place: move the object to "goal"
  - sweep: 
    1) approach to waypoint 1
    2) sweep to waypoint 2
    3) return to home pose

In [13]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType, PlannerConfig

gtimer = GlobalTimer.instance()
gtimer.reset()

In [14]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)

In [15]:
# pick motion - move the object to "gripper"
from_state = initial_state.copy(pscene)
to_node = ("gripper", 0)
for _ in range(10):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    with gtimer.block("plan"):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    pick_state_etasl = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

In [16]:
# place motion - move the object to "goal"
from_state = pick_state_etasl.copy(pscene)
to_node = ("goal", 0)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this placement motion is inherently impossible depending on pick parameter.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

In [17]:
# sweep motion 1 - reach the first waypoint
from_state = initial_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("floor", 1)
for _ in range(3):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=1)
    if success:
        if np.abs(LastQ[0])<np.pi/2: # else, it will fail in sweep motion
            break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep1_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

In [18]:
# sweep motion 2 - sweep "floor" to the second waypoint. Constrained motion planning takes longer time
from_state = sweep1_state.copy(pscene)
to_node = ("floor", 2)
for _ in range(5):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=5)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep2_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this constrained motion is inherently impossible.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

* Interpolated path generation by incremental IK
  - To generate task-space Interpolated path, set MoveitPlanner.incremental_constraint_motion = True
  - If the changing subject returns non-zero constraints with make_constraints, this task-space interpolation will be applied.
  - For simple linear moion, this is fast way to generate constrained motion.
  - This only does linear interpolation and does not observe the constraints.

In [19]:
# Set incremental path generation flag
mplan.incremental_constraint_motion = True

from_state = sweep1_state.copy(pscene)
to_node = ("floor", 2)
for _ in range(5):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=5)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep2_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this constrained motion is inherently impossible.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

## 4.3 TaskPlanner

##### create PlanningPipeline

In [20]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))

In [21]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


##### RRT

In [22]:
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


- pick & place & sweep (single process)
  - this can fail due to short timeout

In [23]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=30, multiprocess=False, timeout=0.5, timeout_constrained=2)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.05/30.0 s, steps/err: 10(45.9449291229 ms)/0.00166115962243)
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->2 (0.09/30.0 s, steps/err: 10(37.6431941986 ms)/0.0013292834391)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->3 (0.19/30.0 s, steps/err: 68(99.7719764709 ms)/0.00225473817709)
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->4 (0.24/30.0 s, steps/err: 10(46.5159416199 ms)/0.00121083482712)
try: 3 - ('gripper', 0)->('goal', 0)
try transition motion
transition motion tried: True
result: 3 - ('gripper', 0)->('goal', 0) 

transition motion tried: False
Motion Plan Failure
result: 19 - ('goal', 0)->('gripper', 0) = fail
try: 5 - ('goal', 0)->('goal', 1)
try transition motion
transition motion tried: True
result: 5 - ('goal', 0)->('goal', 1) = success
branching: 5->21 (4.61/30.0 s, steps/err: 66(116.259813309 ms)/0.00151689619173)
try: 21 - ('goal', 1)->('goal', 2)
try constrained motion
end
constrained motion tried: True
Goal reached
result: 21 - ('goal', 1)->('goal', 2) = success
branching: 21->22 (4.76/30.0 s, steps/err: 41(149.351119995 ms)/0.000102100071149)
try: 2 - ('floor', 1)->('floor', 2)
try constrained motion
end
constrained motion tried: True
result: 2 - ('floor', 1)->('floor', 2) = success
branching: 2->23 (4.91/30.0 s, steps/err: 41(147.686004639 ms)/0.000101799125051)
try: 23 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: True
result: 23 - ('floor', 2)->('gripper', 2) = success
branching: 23->24 (4.95/30.0 s, steps/err: 25(44.4498062134 ms)/0.00128990129667)


end
constrained motion tried: True
Goal reached
result: 42 - ('goal', 1)->('goal', 2) = success
branching: 42->43 (8.93/30.0 s, steps/err: 41(162.487983704 ms)/0.000101824537116)
try: 36 - ('gripper', 0)->('goal', 0)
try transition motion
transition motion tried: True
result: 36 - ('gripper', 0)->('goal', 0) = success
branching: 36->44 (9.0/30.0 s, steps/err: 73(69.1950321198 ms)/0.00150676688687)
try: 44 - ('goal', 0)->('goal', 1)
try transition motion
transition motion tried: True
result: 44 - ('goal', 0)->('goal', 1) = success
branching: 44->45 (9.05/30.0 s, steps/err: 21(44.9080467224 ms)/0.00164299506326)
try: 45 - ('goal', 1)->('goal', 2)
try constrained motion
end
constrained motion tried: True
Goal reached
result: 45 - ('goal', 1)->('goal', 2) = success
branching: 45->46 (9.22/30.0 s, steps/err: 41(168.840169907 ms)/0.000101969793955)
try: 36 - ('gripper', 0)->('gripper', 1)
try transition motion
transition motion tried: True
result: 36 - ('gripper', 0)->('gripper', 1) = succes

joint max
constrained motion tried: False
Motion Plan Failure
result: 34 - ('gripper', 1)->('gripper', 2) = fail
try: 3 - ('gripper', 0)->('gripper', 1)
try transition motion
transition motion tried: True
result: 3 - ('gripper', 0)->('gripper', 1) = success
branching: 3->65 (12.37/30.0 s, steps/err: 62(54.7289848328 ms)/0.00128537631974)
try: 37 - ('floor', 1)->('floor', 2)
try constrained motion
end
constrained motion tried: True
result: 37 - ('floor', 1)->('floor', 2) = success
branching: 37->66 (12.54/30.0 s, steps/err: 41(166.540145874 ms)/0.000101932868966)
try: 66 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: True
result: 66 - ('floor', 2)->('gripper', 2) = success
branching: 66->67 (12.61/30.0 s, steps/err: 43(72.5259780884 ms)/0.00186142545368)
try: 67 - ('gripper', 2)->('goal', 2)
try transition motion
transition motion tried: True
Goal reached
result: 67 - ('gripper', 2)->('goal', 2) = success
branching: 67->68 (12.69/30.0 s, steps/err: 56(73.6

transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->84 (15.74/30.0 s, steps/err: 32(78.663110733 ms)/0.00174705930393)
try: 5 - ('goal', 0)->('gripper', 0)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 5 - ('goal', 0)->('gripper', 0) = fail
try: 3 - ('gripper', 0)->('gripper', 1)
try transition motion
transition motion tried: True
result: 3 - ('gripper', 0)->('gripper', 1) = success
branching: 3->85 (16.33/30.0 s, steps/err: 23(64.4497871399 ms)/0.00187815313066)
try: 56 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 56 - ('floor', 0)->('floor', 1) = success
branching: 56->86 (16.38/30.0 s, steps/err: 25(53.0841350555 ms)/0.00213968254415)
try: 66 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 66 - ('floor', 2)->('gripper', 2) = fail
try: 56 - ('floor', 0)->('gripper', 0)
try transition motion
transition

result: 79 - ('goal', 0)->('goal', 1) = success
branching: 79->103 (20.92/30.0 s, steps/err: 22(28.7480354309 ms)/0.0021043034045)
try: 103 - ('goal', 1)->('goal', 2)
try constrained motion
joint max
constrained motion tried: False
Motion Plan Failure
result: 103 - ('goal', 1)->('goal', 2) = fail
try: 72 - ('gripper', 2)->('floor', 2)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 72 - ('gripper', 2)->('floor', 2) = fail
try: 30 - ('goal', 0)->('goal', 1)
try transition motion
transition motion tried: True
result: 30 - ('goal', 0)->('goal', 1) = success
branching: 30->104 (21.69/30.0 s, steps/err: 11(56.293964386 ms)/0.00191650577012)
try: 104 - ('goal', 1)->('goal', 2)
try constrained motion
end
constrained motion tried: True
Goal reached
result: 104 - ('goal', 1)->('goal', 2) = success
branching: 104->105 (21.84/30.0 s, steps/err: 41(148.474216461 ms)/0.000101816505964)
try: 56 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tr

result: 84 - ('gripper', 0)->('floor', 0) = fail
try: 56 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 56 - ('floor', 0)->('floor', 1) = success
branching: 56->124 (27.03/30.0 s, steps/err: 25(89.0650749207 ms)/0.00182413132737)
try: 96 - ('gripper', 0)->('goal', 0)
try transition motion
transition motion tried: True
result: 96 - ('gripper', 0)->('goal', 0) = success
branching: 96->125 (27.12/30.0 s, steps/err: 69(81.3930034637 ms)/0.00147365087849)
try: 125 - ('goal', 0)->('goal', 1)
try transition motion
transition motion tried: True
result: 125 - ('goal', 0)->('goal', 1) = success
branching: 125->126 (27.16/30.0 s, steps/err: 27(36.9148254395 ms)/0.00145108928074)
try: 126 - ('goal', 1)->('goal', 2)
try constrained motion
end
constrained motion tried: True
Goal reached
result: 126 - ('goal', 1)->('goal', 2) = success
branching: 126->127 (27.32/30.0 s, steps/err: 41(164.796829224 ms)/0.000102025653343)
try: 66 - ('floor', 2)->('gripper', 2)


IndexError: list index out of range

- pick & place & sweep (multi process)

In [24]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=30, multiprocess=True, timeout=0.5, timeout_constrained=2)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

Use 36/36 agents
try: 0 - ('floor', 0)->('gripper', 0)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
transition motion tried: True
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
try transition motion
branching: 0->1 (0.16/30.0 s, steps/err: 10(54.8121929169 ms)/0.000762557113128)
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->2 (0.17/30.0 s, steps/err: 10(88.9010429382 ms)/0.000762557113128)
try transition motion
transition mo

branching: 15->20 (0.78/30.0 s, steps/err: 48(101.472854614 ms)/0.00154193048094)
try constrained motion
try: 0 - ('floor', 0)->('floor', 1)
try: 19 - ('gripper', 2)->('goal', 2)
try: 20 - ('gripper', 2)->('goal', 2)
try transition motion
try transition motion
try transition motion
try: 11 - ('gripper', 0)->('gripper', 1)
transition motion tried: False
try transition motion
Motion Plan Failure
try: 17 - ('gripper', 0)->('gripper', 1)
transition motion tried: True
try: 0 - ('floor', 0)->('floor', 1)
try: 17 - ('gripper', 0)->('floor', 0)
try: 13 - ('gripper', 0)->('goal', 0)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 9 - ('gripper', 0)->('floor', 0)
try transition motion
try transition motion
try: 16 - ('goal', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
try transition motion
try transition motion
branching: 0->21 (0.84/30.0 s, steps/err: 10(48.2180118561 ms)/0.000762557113128)
transition m

try: 15 - ('floor', 2)->('gripper', 2)
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
end
try: 34 - ('gripper', 1)->('gripper', 2)
Motion Plan Failure
branching: 0->37 (1.17/30.0 s, steps/err: 10(84.8670005798 ms)/0.00119880262738)
try: 40 - ('goal', 1)->('goal', 2)
transition motion tried: True
try transition motion
result: 7 - ('gripper', 0)->('gripper', 1) = success
result: 27 - ('goal', 0)->('goal', 1) = success
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = success
result: 14 - ('floor', 2)->('gripper', 2) = fail
transition motion tried: True
try constrained motion
constrained motion tried: True
branching: 7->39 (1.18/30.0 s, steps/err: 57(165.971040726 ms)/0.00183205714517)
result: 31 - ('goal', 0)->('goal', 1) = success
try constrained motion
branching: 27->40 (1.19/30.0 s, steps/err: 35(65.169095993 ms)/0.0019150223574)
result: 27 - ('goal', 0)->('goal', 1) = success
branching: 0->38 (1.19/30.0 s, steps/err: 10(100.8

result: 0 - ('floor', 0)->('floor', 1) = success
try: 17 - ('gripper', 0)->('floor', 0)
try: 0 - ('floor', 0)->('floor', 1)
result: 9 - ('gripper', 0)->('floor', 0) = fail
result: 0 - ('floor', 0)->('gripper', 0) = fail
branching: 0->55 (1.54/30.0 s, steps/err: 10(103.99389267 ms)/0.00157364280083)
try transition motion
transition motion tried: False
try transition motion
transition motion tried: True
try transition motion
Motion Plan Failure
transition motion tried: True
try: 11 - ('gripper', 0)->('floor', 0)
result: 45 - ('goal', 0)->('goal', 1) = success
try transition motion
transition motion tried: False
result: 12 - ('gripper', 0)->('goal', 0) = success
result: 17 - ('gripper', 0)->('floor', 0) = fail
try: 56 - ('goal', 1)->('goal', 2)
try: 45 - ('goal', 0)->('goal', 1)
Motion Plan Failure
result: 15 - ('floor', 2)->('gripper', 2) = fail
try: 57 - ('goal', 0)->('goal', 1)
try transition motion
transition motion tried: False
branching: 12->57 (1.59/30.0 s, steps/err: 90(154.788970

try: 0 - ('floor', 0)->('floor', 1)
transition motion tried: True
result: 34 - ('gripper', 1)->('gripper', 2) = success
result: 42 - ('gripper', 0)->('goal', 0) = fail
try transition motion
try transition motion
result: 0 - ('floor', 0)->('floor', 1) = success
try: 73 - ('gripper', 2)->('goal', 2)
try: 0 - ('floor', 0)->('floor', 1)
try: 27 - ('goal', 0)->('gripper', 0)
try transition motion
joint max
branching: 0->74 (1.87/30.0 s, steps/err: 10(63.4560585022 ms)/0.00136703052167)
try constrained motion
constrained motion tried: False
try transition motion
Goal reached
transition motion tried: False
Motion Plan Failure
branching: 34->73 (1.87/30.0 s, steps/err: 41(664.593935013 ms)/0.000101953966176)
transition motion tried: True
Motion Plan Failure
try: 31 - ('goal', 0)->('gripper', 0)
result: 48 - ('goal', 1)->('goal', 2) = success
try: 0 - ('floor', 0)->('gripper', 0)
branching: 48->75 (1.89/30.0 s, steps/err: 41(530.946969986 ms)/0.000101953966176)
transition motion tried: True
tra

...

try transition motion
try transition motion
branching: 174->474 (7.28/30.0 s, steps/err: 40(75.5031108856 ms)/0.00109143756074)
try: 58 - ('gripper', 0)->('gripper', 1)
transition motion tried: True
branching: 109->476 (7.29/30.0 s, steps/err: 35(76.593875885 ms)/0.00172658692318)
try: 476 - ('goal', 0)->('goal', 1)
transition motion tried: True
result: 158 - ('floor', 0)->('gripper', 0) = fail
try transition motion
end
try: 32 - ('gripper', 0)->('gripper', 1)
result: 241 - ('gripper', 2)->('floor', 2) = fail
try transition motion
result: 400 - ('gripper', 0)->('gripper', 1) = success
try: 84 - ('floor', 1)->('floor', 2)
try: 232 - ('gripper', 0)->('goal', 0)
constrained motion tried: True
try transition motion
transition motion tried: False
try: 83 - ('goal', 0)->('goal', 1)
result: 266 - ('gripper', 0)->('gripper', 1) = success
try: 186 - ('gripper', 0)->('gripper', 1)
Motion Plan Failure
try transition motion
try transition motion
branching: 400->477 (7.32/30.0 s, steps/err: 74(111.

transition motion tried: True
Motion Plan Failure
transition motion tried: True
branching: 65->494 (7.53/30.0 s, steps/err: 61(130.95498085 ms)/0.00134444310443)
result: 140 - ('gripper', 0)->('floor', 0) = fail
result: 222 - ('gripper', 0)->('goal', 0) = success
transition motion tried: True
result: 242 - ('goal', 0)->('goal', 1) = success
try: 494 - ('goal', 0)->('goal', 1)
branching: 471->492 (7.54/30.0 s, steps/err: 41(306.891918182 ms)/0.000101888574553)
branching: 222->496 (7.55/30.0 s, steps/err: 21(58.2020282745 ms)/0.00149764948387)
branching: 242->495 (7.54/30.0 s, steps/err: 38(421.650886536 ms)/0.00169130455775)
try: 493 - ('gripper', 2)->('goal', 2)
try transition motion
transition motion tried: False
++ adding return motion to acquired answer ++
try transition motion
transition motion tried: True
try: 496 - ('goal', 0)->('goal', 1)
transition motion tried: True
try: 495 - ('goal', 1)->('goal', 2)
Motion Plan Failure
result: 105 - ('gripper', 0)->('gripper', 1) = success
t

collision
branching: 12->512 (7.73/30.0 s, steps/err: 68(85.7608318329 ms)/0.0018617939807)
try transition motion
constrained motion tried: False
try: 511 - ('gripper', 2)->('goal', 2)
Motion Plan Failure
result: 232 - ('gripper', 0)->('floor', 0) = fail
Goal reached
Motion Plan Failure
try transition motion
transition motion tried: True
result: 488 - ('goal', 1)->('goal', 2) = success
result: 507 - ('goal', 1)->('goal', 2) = fail
result: 489 - ('goal', 0)->('goal', 1) = success
Goal reached
transition motion tried: False
branching: 488->513 (7.77/30.0 s, steps/err: 41(289.025068283 ms)/0.000101901664113)
transition motion tried: True
try transition motion
transition motion tried: True
Motion Plan Failure
branching: 489->514 (7.77/30.0 s, steps/err: 49(298.44212532 ms)/0.00167454127992)
result: 348 - ('gripper', 0)->('gripper', 1) = success
try: 514 - ('goal', 1)->('goal', 2)
result: 115 - ('goal', 0)->('gripper', 0) = fail
try: 441 - ('goal', 0)->('goal', 1)
try constrained motion
try

++ adding return motion to acquired answer ++
Goal reached
======================= terminated 25: Stop called from other agent  (7.9/30.0) ===============================
++ adding return motion to acquired answer ++
======================= terminated 29: Stop called from other agent  (8.0/30.0) ===============================
transition motion tried: False
transition motion tried: False
Motion Plan Failure
result: 482 - ('floor', 2)->('gripper', 2) = fail
Motion Plan Failure
======================= terminated 30: Stop called from other agent  (8.0/30.0) ===============================
result: 0 - ('floor', 0)->('gripper', 0) = fail
======================= terminated 18: required answers acquired  (8.0/30.0) ===============================
======================= terminated 9: Stop called from other agent  (8.0/30.0) ===============================
Goal reached
Goal reached
end
constrained motion tried: True
Goal reached
result: 502 - ('goal', 1)->('goal', 2) = success
branching: 502->

transition motion tried: False
Motion Plan Failure
result: 105 - ('gripper', 0)->('floor', 0) = fail
======================= terminated 13: Stop called from other agent  (8.2/30.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 241 - ('gripper', 2)->('floor', 2) = fail
======================= terminated 21: Stop called from other agent  (8.2/30.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 301 - ('floor', 0)->('gripper', 0) = fail
======================= terminated 6: Stop called from other agent  (8.2/30.0) ===============================
======================= terminated 5: required answers acquired  (8.2/30.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 301 - ('floor', 0)->('gripper', 0) = fail
======================= terminated 28: Stop called from other agent  (8.3/30.0) ===============================
transition motion tried: False
Motion Pla

##### PDDLStream
* pick & place
* currently only *single robot* is supported
* only colliding geometries are transfered to PDDLStream(pybullet). make sure collision=True for all subjects and actors.

In [22]:
## Remove sweep - currently not supported with PDDLStream
pscene.remove_subject("sweep")
pscene.remove_binder("sweep_face")
initial_state = pscene.initialize_state(crob.home_pose)

   Use a production WSGI server instead.
 * Debug mode: off


In [23]:
from pkg.planning.pddlstream.plan_rnb import solve_in_pddlstream, play_pddl_plan

In [24]:
goal_pairs = [('box1', 'goal')]
res, plan, log_dict = solve_in_pddlstream(pscene, mplan, 'indy0', 'grip0', crob.home_pose, goal_pairs,
                                          TIMEOUT_MOTION=0.5, MAX_TIME=30, MAX_ITER=100, MAX_SKELETONS=10,
                                          GRASP_SAMPLE=30, STABLE_SAMPLE=50, SHOW_STATE=False, SEARCH_SAMPLE_RATIO=10,
                                          use_pybullet_gui=False, USE_MOVEIT_IK=True)

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
Objects: {1L: 'goal', 2L: 'gripper', 3L: 'floor', 4L: 'wall', 5L: 'obstacle', 6L: 'box1'}
================ MAKE PROBLEM ======================
IK checkers: []
MP checkers: []
timeout motion : 0.5
Robot: 0
Movable: [6L]
Fixed: [1, 2, 3, 4, 5]
body 6 - surface 1
body 6 - surface 2
body 6 - surface 3
body 6 - surface 4
body 6 - surface 5
[INFO] Update stable set for box1 - goal
[INFO] Update stable set for box1 - floor
[INFO] Update grasp set for box1
Init: [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Timer', 0L), ('Graspable', 6L), ('Pose', 6L, p0), ('AtPose', 6L, p0), ('Stackable', 6L, 1), ('Stackable', 6L, 2), ('Stackable', 6L, 3), ('Supported', 6L, p0, 3), ('Stackable', 6L, 4), ('Stackable', 6L, 5)]
Goal: ('and', ('AtConf', q0), ('On', 6L, 1L))
Streams: {TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approac

In [25]:
play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
               body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)